<a href="https://colab.research.google.com/github/mariaclaraseixa/corpo-metallico-cma/blob/main/resultado_metal_cma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando a tabela do excel com todos os valores obtido pela maquina durante o ensaio

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [18]:


L0 = 70.0      # L0 em mm
Lf = 91.86     # Lf final em mm (medido no paquímetro)

# Arquivo de dados do ensaio (ajuste o nome/caminho se necessário)
arquivo_excel = "Book 2.xlsx"


df = pd.read_excel(arquivo_excel)

# print(df.columns)

col_desloc = "Deslocamento"   # ΔL em mm
col_tensao = "Tensão.1"       # σ em MPa

desloc = df[col_desloc].values.astype(float)   # ΔL (mm)
sigma  = df[col_tensao].values.astype(float)   # tensão (MPa)

print(f"Comprimento inicial L0: {L0:.2f} mm")
print(f"Comprimento final   Lf: {Lf:.2f} mm\n")

Comprimento inicial L0: 70.00 mm
Comprimento final   Lf: 91.86 mm



# Deformação ruptura

In [19]:
eps_f = (Lf - L0) / L0          # deformação adimensional
eps_f_percent = eps_f * 100.0   # em %

print(f"   εf = (Lf - L0) / L0 = {eps_f:.5f} mm/mm")
print(f"   εf = {eps_f_percent:.2f} %\n")


   εf = (Lf - L0) / L0 = 0.31229 mm/mm
   εf = 31.23 %



# Tensão Máxima (≈480.20 MPa)

In [20]:
sigma_max = np.max(sigma)
idx_max   = np.argmax(sigma)
desloc_max = desloc[idx_max]

In [21]:
print("Tensão Máxima / UTS:")
print(f"   σ_max = {sigma_max:.2f} MPa")
print(f"   Ocorre em ΔL ≈ {desloc_max:.4f} mm\n")

Tensão Máxima / UTS:
   σ_max = 480.20 MPa
   Ocorre em ΔL ≈ 15.6088 mm



# Módulo de Elasticidade (≈22.16 GPa)

In [22]:
#  Regressão linear na região elástica (ex: ΔL < 0,3 mm)

limite_elastico_desloc = 0.3  # mm (critérios de seleção da região elástica)
mask_elastico = desloc <= limite_elastico_desloc

x_elast = desloc[mask_elastico]
y_elast = sigma[mask_elastico]

# Regressão linear: σ = m * ΔL + b
m, b = np.polyfit(x_elast, y_elast, 1)

# Pela dedução: E = m * L0
E = m * L0             # MPa
E_GPa = E / 1000.0     # GPa

In [23]:
print("Módulo de Elasticidade (Módulo de Young):")
print(f"   Regressão linear em ΔL <= {limite_elastico_desloc} mm")
print(f"   σ = m * ΔL + b  → m = {m:.4f} MPa/mm ; b = {b:.4f} MPa")
print(f"   E = m * L0 = {E:.2f} MPa ≈ {E_GPa:.2f} GPa\n")


Módulo de Elasticidade (Módulo de Young):
   Regressão linear em ΔL <= 0.3 mm
   σ = m * ΔL + b  → m = 316.5722 MPa/mm ; b = 156.8117 MPa
   E = m * L0 = 22160.06 MPa ≈ 22.16 GPa



# Tenacidade em tração (≈145 MPa)

Integra numérica da curva σ x ΔL (trapézios) e / pelo comp. inicial para obter MJ/m³

In [24]:
# Área sob σ x ΔL (MPa * mm)
area_sigma_desloc = np.trapezoid(sigma, desloc)

# Tenacidade = área / L0  → unidade final: MPa = MJ/m³
tenacidade = area_sigma_desloc / L0  # MPa ou MJ/m³


In [25]:
print("tenacidade em tração:")
print(f"   Área sob a curva σ x ΔL ≈ {area_sigma_desloc:.2f} MPa·mm")
print(f"   Tenacidade = área / L0 ≈ {tenacidade:.2f} MPa (MJ/m³)")
print(f"  U ≈ {tenacidade:.2f} MJ/m³\n")


tenacidade em tração:
   Área sob a curva σ x ΔL ≈ 10163.89 MPa·mm
   Tenacidade = área / L0 ≈ 145.20 MPa (MJ/m³)
  U ≈ 145.20 MJ/m³



# Limite de escoamento (deformação ≈ 1.93 %, tensão  ≈ 1.3489 mm e deslocamento 1.3489 mm)

**método da linha offset de 0,2%**

In [26]:
# Primeiro, calculamos a deformação engenheirada ε = ΔL / L0
eps = desloc / L0

# Reta offset 0,2%: σ_offset(ε) = E * (ε - 0,002)
offset = 0.002  # 0,2% de deformação
sigma_offset_line = E * (eps - offset)

# Para achar a interseção experimental x offset:
# buscamos mudança de sinal em (σ_exp - σ_offset)
diff = sigma - sigma_offset_line

# Encontrar o primeiro ponto em que o sinal muda
yield_index = None
for i in range(1, len(diff)):
    if diff[i-1] * diff[i] < 0:
        yield_index = i
        break

if yield_index is not None:
    # interpolação linear entre os pontos (i-1, i)
    x1, x2 = eps[yield_index-1], eps[yield_index]
    y1, y2 = diff[yield_index-1], diff[yield_index]

    # fração para interpolar o zero de diff
    frac = -y1 / (y2 - y1)

    eps_y = x1 + frac * (x2 - x1)        # deformação no escoamento
    sig_y = E * (eps_y - offset)         # tensão correspondente (reta offset)
    desloc_y = eps_y * L0                # ΔL no escoamento
else:
    eps_y = np.nan
    sig_y = np.nan
    desloc_y = np.nan

In [27]:

print("Limite de escoamento:")
if not np.isnan(sig_y):
    print(f"   Deformação no escoamento ε_y ≈ {eps_y:.5f} (≈ {eps_y*100:.2f} %)")
    print(f"   Deslocamento no escoamento ΔL_y ≈ {desloc_y:.4f} mm")
    print(f"   Tensão de escoamento σ_y ≈ {sig_y:.2f} MPa")
else:
    print("   Não foi possível determinar o ponto de escoamento com o critério implementado.")

Limite de escoamento:
   Deformação no escoamento ε_y ≈ 0.01927 (≈ 1.93 %)
   Deslocamento no escoamento ΔL_y ≈ 1.3489 mm
   Tensão de escoamento σ_y ≈ 382.71 MPa
